In [1]:
import sys
!{sys.executable} -m pip install kfp >/dev/null

You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
import os
import kfp

In [3]:
dkube_training_op           = kfp.components.load_component_from_url("https://raw.githubusercontent.com/oneconvergence/dkube-training/main/components/training/component.yaml")
dkube_serving_op            = kfp.components.load_component_from_url("https://raw.githubusercontent.com/oneconvergence/dkube-training/main/components/serving/component.yaml")

In [7]:
@kfp.dsl.pipeline(
    name='dkube-mnist-pl',
    description='sample mnist pipeline with dkube components'
)
def mnist_pipeline(token, program='training'):

    train       = dkube_training_op(token, '{"image":"ocdr/dkube-datascience-tf-cpu:v2.0.0"}',
                                    framework="tensorflow", version="2.0.0",
                                    program="training", run_script="python mnist/train.py",
                                    datasets='["training"]', outputs='["training"]',
                                    input_dataset_mounts='["/mnist"]',
                                    output_mounts='["/opt/dkube/output"]',
                                    envs='[{"EPOCHS": "5"}]')

    serving     = dkube_serving_op(token, train.outputs['artifact'], device='cpu', 
                                    serving_image='{"image":"ocdr/tensorflowserver:2.0.0"}',
                                    transformer_image='{"image":"ocdr/dkube-datascience-tf-cpu:v2.0.0"}',
                                    transformer_project="training",
                                    transformer_code='mnist/transformer.py').after(train)

In [8]:
token  = os.getenv("DKUBE_USER_ACCESS_TOKEN")
client = kfp.Client(existing_token=token)
client.create_run_from_pipeline_func(mnist_pipeline, arguments={"token":token})

RunPipelineResult(run_id=8cef92f3-f875-4b2f-945c-988cdcfea2eb)

In [9]:
#generate & upload pipeline (Optional)
import kfp.compiler as compiler
compiler.Compiler().compile(mnist_pipeline, "mnist-pipeline.zip")
client.upload_pipeline("mnist-pipeline.zip")

{'created_at': datetime.datetime(2020, 12, 16, 22, 50, 16, tzinfo=tzlocal()),
 'default_version': {'code_source_url': None,
                     'created_at': datetime.datetime(2020, 12, 16, 22, 50, 16, tzinfo=tzlocal()),
                     'id': '3432e270-ce8a-4a88-97a6-bc14aa0e1ce3',
                     'name': 'mnist-pipeline.zip',
                     'package_url': None,
                     'parameters': [{'name': 'token', 'value': None},
                                    {'name': 'program', 'value': 'training'}],
                     'resource_references': [{'key': {'id': '3432e270-ce8a-4a88-97a6-bc14aa0e1ce3',
                                                      'type': 'PIPELINE'},
                                              'name': None,
                                              'relationship': 'OWNER'}]},
 'description': None,
 'error': None,
 'id': '3432e270-ce8a-4a88-97a6-bc14aa0e1ce3',
 'name': 'mnist-pipeline.zip',
 'parameters': [{'name': 'token', 'value': N